In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None
from IPython.core.display import HTML

HTML(open("custom.html", "r").read())

<p style="font-size: 2.5em; font-weight: bold;">Section 4: Optimization</p>

In this section we will introduce important libraries and  tools speed up code without parallelization. 

**These optimizations will not reduce the runtime complexity $\mathcal{O}(...)$ but the involved (hidden) constants!**

<table>
    <tr><td><img src="images/overview-hpc.svg" alt="overview" width="600px"></td></tr>
    <tr><td><center><sub>(c) ETH Zurich</sub></center></td></tr>
</table>

# NumPy, SciPy, NumExpr

Before we discuss how to use NumPy, SciPy and NumExpr to speed up code we introduce an example which we will use in many places during this workshop:

## Computing $\pi$ using a Monte Carlo simulation

The idea to compute an approximation to $\pi$ is to generate $n$ uniformly distributed random $(x, y)$ points in a $2D$ square of side length $2$, centered at the origin. 

Imagine a circle inscribed into the square, i.e. the unit circle. 

<table>
    <tr><td><img src="images/monte_carlo_pi.png" width="450px" ></td></tr>
   <tr><td><center><sub>Source: <a href="https://www.kaggle.com/nickgould/monte-carlo-tutorial-calculating-pi/comments">https://www.kaggle.com/nickgould/monte-carlo-tutorial-calculating-pi/comments</a></sub></center></td></tr>
</table>


The probability to generate a random point from the square in which is also located in the cirle is

$$p = \frac{A_{\text{circle}}}{A_{\text{square}}}$$

$A_{\text{circle}}$: Area of the circle,

$A_{\text{square}}$: Area of the square.

We have a circle of radius $r = 1$, enclosed by a 2×2 square. The area of the circle is $\pi r^2=\pi$, the area of the square is $4$. Thus

<math>\begin{align}
p = \frac{\pi}{4}
\end{align}
</math>

Hence if we generate $n$ random points in the square and count $n_{\text{inner}}$ points which are located in the circle:

<math>\begin{align}
\pi \approx  4 \frac{n_{\text{inner}}}{n} 
\end{align}
</math>

To check if a point $(x, y)$ is located within the circle, we can use the the Euclidean formula to compute the distance $\text{dist}$ to $(0, 0)$ and check if this distance is $\le 1$:

$$
\text{dist}(x, y) = \sqrt{x^2 + y^2} \le 1
$$

We can square to both sides to avoid computing the square root and get

$$
\text{dist}^2(x, y) = x^2 + y^2 \le 1
$$



Let us simulate this:

In [ ]:
from random import uniform


def approx_pi(n_attempts):
    n_hits = 0

    for _ in range(n_attempts): # _ denotes a unused variable
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        # check if (x, y) lies in the cirle
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts


# check if it works
print(approx_pi(2_000_000))

# run the timings
%timeit approx_pi(2_000_000)

`for` loops are slow 🐌 in 🐍! In each iteration the whole body is reinterpreted which costs time.

One possibility to optimize is to reduce the amount of code and / or use [generator expressions](https://www.python.org/dev/peps/pep-0289/).

What are generator expressions?

> A high performance, memory efficient generalization of list comprehensions and generators.

Source: [PEP 289](https://www.python.org/dev/peps/pep-0289/#id13)

Example: `(i**2 for i in range(n))`

could be used wherever an iterable is accepted. In contrast to a list comprehension (see [Section 3, Appending an element to an array](../section_3/Section&nbsp;3.ipynb#Appending-an-element-to-an-array)), no temporary list is allocated in memory - the items are lazily generated on demand. Therefore they behave exactly the same as generators (see [Section 3, Generators](../section_3/Section&nbsp;3.ipynb#Generators)).

In [ ]:
from random import uniform


def approx_pi(n_attempts):
    return (
        4
        / n_attempts
        * sum(
            uniform(-1.0, 1.0) ** 2 + uniform(-1.0, 1.0) ** 2 <= 1.0
            for _ in range(n_attempts)
        )
    )


# check if it works:
print(approx_pi(2_000_000))

# run the timings
%timeit approx_pi(2_000_000)

But this does not help much.
To gain more performance, we can outsource code to C / Fortran using NumPy:
Here, we rewrite the above 🐍 code using NumPy:

In [ ]:
import numpy as np


def approx_pi(n_attempts):
    # n_attempts x 2 matrix:
    points = np.random.rand(n_attempts, 2)
    # square values, sum horizontally and count hits:
    n_hits = np.count_nonzero((points ** 2).sum(axis=1) <= 1.0)
    return 4 * n_hits / n_attempts


# check if it works:
print(approx_pi(2_000_000))

%timeit approx_pi(2_000_000)

## Why is numpy faster than pure Python?

* Written in C / Fortran (loops are outsourced to low level programming languages)
* Has an own array data type containing homogeneous types (optimized memory layout)

<table>
    <tr><td><img src="images/numpy_memory.svg" width="800px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://www.nature.com/articles/s41586-020-2649-2/figures/1">www.nature.com/articles/s41586-020-2649-2/figures/1</a></sub></center></td></tr>
</table>

In contrast, here is the same realized using pure Python data structures:

<table>
    <tr><td><img src="images/python_memory.svg" alt="py array memeory layout" width="800px"></td></tr>
    <tr><td><center><sub>(c) ETH Zurich</sub></center></td></tr>
</table>

<div class="alert alert-block alert-success">
<b>Note:</b> <i>pandas</i> is built on top of NumPy's data structures / code.
</div>

## What else can we use to increase performance?

Basically there are 2 approaches:
* Stick to pure Python / optimize the interpreter (this will lead us to the concept of Just-in-time compiling, see later in the script)
* Use a low level compiled programming language directly or indirectly (like NumPy / `pandas`)

In order to understand these two approaches, we introduce the crucial concept of source code compilation.

**Compilation** is a process of translating computer code written in a developer-friendly high-level *source programming language* into another a machine-friendly low-level *target programming language*. Compilation is done by a program called *compiler*.

<table>
    <tr><td><img src="images/1000px-Compiler_Shematic.png" width="800px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://hpc-wiki.info/hpc/Compiler">https://hpc-wiki.info/hpc/Compiler</a></sub></center></td></tr>
</table>

The compiled program is optimized for the target machine to run fast on it, but be aware that the compilation itself may take quite some time and memory - the bigger the source program the longer it takes to compile it.

The tradeoff when using compiled languages is: performance / flexibility in exchange for portability / usability.

### Tradeoff: Performance / flexibility vs. portability

**Performance:** When using a low level programming language, we have full control over memory management and full **flexibility** while developing algorithms. The backside is, that the amount of code will also increase drastically when compared with Python (**usability**).

**Portability:** 
When writing code in a foreign language to be called from Python code, this will affect how to distribute your code:

- This could be e.g. distributing a binary or source package via pypi.
- A binary package (also called **wheel**) does not require a working C/C++/Fortran compiler on the end users computer.
   - Such a package is built on one machine and may not use the target machines setup at maximum efficiency. 
   - You must build different wheels for each target OS, such as Linux, Mac OS X and Windows.
- A source distribution by contrast can be optimized for the target platform but requires a working development environment including compilers. 
   - This is especially cumbersome on Windows computers. 
   - Windows does not ship with a C/C++ compiler and the compiler version must match the Python version.
   - See also [these instructions](https://pythondev.readthedocs.io/windows.html#python-and-visual-studio-version-matrix).
    

## numpy arrays in memory

A NumPy array contains elements of a uniform data type, i.e. the same size for each data item, and is stored densely in a contiguous block of memory. 

In [ ]:
import numpy as np

arr = np.array([[0, 1, 2], [3, 4, 5]])
print(arr)
print(arr.dtype)

In the computer's memory, a (possibly multidimensional) array `arr` is stored as a single contiguous block. 

To flatten a 2 dimensional array there are two common methods:
- `C` contiguous: flatten row per row
- `F` (Fortran) contiguous: flatten column per column.

<img src="images/contiguous_memory.png" width=50%/>

**C contiguous is used in NumPy per default**: Neighbors of elements within a row are neighbors in memory. In memory, the last element of a row is a neighbor of the first element of the next row. 

CPUs work more efficiently when they access close memory locations (remember `L1`, `L2`, `L3` caching from section 1)?

In [ ]:
arr = np.random.rand(10_000, 10_000)

# sum up first row, no need to jump in memory:
%timeit arr[0, :].sum()

# sum up first column, we have to jump in memory in steps of 10_000:
%timeit arr[:, 0].sum()

To create an array stored in a Fortran contiguous block, you can specify the option `order='F'` when creating the array. `.flags` provides the information about the array's memory layout.

In [ ]:
arr = np.array([[1, 2, 3], [4, 5, 6]], order="F") # F for "Fortran contigous"
print(arr)
arr.flags

In [ ]:
arr = np.random.rand(10_000, 10_000)
arr = np.asfortranarray(arr)

# sum up first row, we have to jump in memory in steps of 10_000:
%timeit arr[0, :].sum()

# sum up first column, no need to jump in memory:
%timeit arr[:, 0].sum()

## Vectorization and Universal functions (ufuncs)

NumPy uses so called [ufuncs](https://numpy.org/doc/stable/reference/ufuncs.html) (universal functions) to make the repeated calculations on array elements more efficient and therefore faster. This process is also referred to as **vectorization**. For mathematical operations between arrays with the same dimension, NumPy applies the operation element-wise.

With vectorization we can avoid an explicit iteration over the array, e.g., for-loop in Python, which is rather slow. Instead the loop runs in C.

Let's compare the `sum` operation in Python and NumPy.

In [ ]:
arr = np.arange(10000)

%timeit np.sum(arr)
%timeit sum(arr)

From the time measurement on my computer, `np.sum` is much faster than the plain Python sum function!


Let's compare the `np.sin` `ufunc` function from NumPy to a plain Python implementation:

In [ ]:
def apply_sin(x):
    result = np.zeros_like(x)
    m, n = result.shape
    for i in range(m):
        for j in range(n):
            result[i, j] = np.sin(x[i, j])
    return result


# 100 x 100 matrix
n = 100
# In reshape, one shape dimension can be -1.
# In this case, the value is inferred from the length of
# the array and remaining dimensions:
arr = np.arange(n * n, dtype=np.float64).reshape(n, -1)

%timeit apply_sin(arr)
%timeit np.sin(arr)

- On my computer this is a speed-up by a factor of ~150
- The NumPy version is also much shorter and more readable!

### Performance of different floating point types

<div class="alert alert-block alert-warning">
  ⚠ The following is highly hardware dependent!
</div>

We now compare the performance of `numpy.sin` for numpy floats of `128`, `64`, `32`, `16`:

In [ ]:
import numpy as np

for dtype in (np.float128, np.float64, np.float32, np.float16):
    # a bit complicated way to create an array but avoids overflows:
    data = 0.001 * np.arange(1000, dtype=dtype).repeat(100)
    print(dtype)
    print("sum")
    %timeit np.sum(data)
    print("sin")
    %timeit np.sin(data)
    print()

## Broadcasting

Broadcasting allows arithmetic operations in an efficient way between two arrays with different shapes or an array with a scalar. 

Similar to the `ufuncs` this can reduce writing Python loops which will run on C-level instead.

This is broadcasting: we add a scalar to a NumPy array:

In [ ]:
arr = np.arange(6).reshape(3, -1)
print(arr)

In [ ]:
print(1 + arr)

And this also: We add a vector of shape `(3, 1)` to an array of shape `(3, 2)`:

In [ ]:
# add same vector to each column:
b = np.arange(3).reshape(3, 1)
print(b, "+", arr, "=", b + arr)

Or we can multiply a vector of shape `(1, 2)` with the array or shape `(3, 2)`:

In [ ]:
c = np.arange(1, 3).reshape(1, 2)
print(c, "*", arr, "=", c * arr)

More about broadcasting here: https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html



## Memory allocation vs. in-place operations

You have seen that using the NumPy array and its operations is efficient and, therefore, fast. However, it is possible to write an un-optimized code with NumPy as well.

Let us look at the following two examples:

In [ ]:
# First example
arr = np.array([[0, 1, 2], [3, 4, 5]])
arr = arr * 2
arr

In [ ]:
# Second example
arr = np.array([[0, 1, 2], [3, 4, 5]])
arr *= 2
arr

<div class="alert alert-block alert-info">
    <p style="font-weight: bold; font-size:120%;"><i class="fa fa-question-circle"></i>&nbsp; Question to the audience</p>

What do you think is the difference between the above 2 operations?

</div>

The difference of the two example is that memory allocation only happens in the first example. 

When Python executes`arr = arr * 2`
1. a new array for the result is created
2. numbers in `arr` are multiplied by `2` and copied the the new array,
3. the name `arr` is set to point to the new array
4. eventually the data from the initial `arr` is deleted.


In the second example, `arr *= 2` is an in-place operation. The only operation is

1. numbers in `arr` are replaced by their doubled value

So you can see: in-place multiplication

1. does not spend time in memory management
2. avoids the temporary increase of memory consumption by a factor of 2

To check if two arrays are sharing the same data buffer, we use `np.shares_memory`. As expected, the input and output arrays do not share the same data location in the first example, while there is no array copying in the second example and the data location remains the same after the in-place operation.

In [ ]:
# First example: operation with array copying
arr = np.array([[0, 1, 2], [3, 4, 5]])
b = arr * 2
np.shares_memory(arr, b)

In [ ]:
# Second example: in-place operation
arr = np.array([[0, 1, 2], [3, 4, 5]])
b = arr  # Create another pointer to the original data buffer
arr *= 2
np.shares_memory(arr, b)

In this section, we will take a look, when memory allocation happens and when not. When we understand this, we can avoid unnecessary array copying and reduce memory use during the computation.

### View vs. copy

Another example of an in-place operation is again slicing, e.g., `arr[i:j]` which is a view of the array `arr` and only points to the data buffer (not a copy).

In [ ]:
arr = np.array([[0, 1, 2], [3, 4, 5]])
b = arr

print("b and arr before:", b)

b[0, 1] = 888
print("\nb, arr after:", b, arr)

To explicitly copy an array, the easiest option is to use `.copy()`. Note that this function has `order='C'` by default. Therefore, unless specified, `.copy()` returns the output array with C-order regardless the memory structure of the input array.

In [ ]:
arr = np.array([[0, 1, 2], [3, 4, 5]], order="F")
b = arr.copy()

print("b and arr before:", b)

b[0, 1] = 888

print("\nb, arr after:", b, arr)

`np.copy()` is similar to `.copy()` but the default order is `'K'` which means match the layout of the input array.

In [ ]:
arr = np.array([[0, 1, 2], [3, 4, 5]], order="F")
b = np.copy(arr)

print("b and arr before:", b)

b[0, 1] = 888

print("\nb, arr after:", b, arr)

### Advanced indexing

We have learned that indexing and slicing are only views of an array. However, advanced indexing with integer or boolean returns always a copy of the data.

In [ ]:
arr = np.array(
    [
        [0, 1, 2],
        [3, 4, 5],
    ]
)

# extract elements (0, 0), (1, 0) and (1, 2):
b = arr[[0, 1, 1], [0, 0, 2]]

print(arr)
print(b)
np.shares_memory(arr, b)

In [ ]:
arr = np.array(
    [
        [0, 1, 2],
        [3, 4, 5],
    ]
)
b = arr[arr > 2]
print("arr > 2: ", arr > 2)
print("arr =", arr)
print("b =", b)
np.shares_memory(arr, b)

### Reshaping

<div class="alert alert-block alert-info">
    <p style="font-weight: bold; font-size:120%;"><i class="fa fa-question-circle"></i>&nbsp; Question to the audience</p>

What about the reshape operation? Does it need array copying?

</div>

The following examples show reshaping an array, transposing an array and reshaping a transposed array. 

In [ ]:
arr = np.array([[0, 1, 2], [3, 4, 5]])
b = arr.reshape(1, -1) # Second dimension automatically inferred
print(arr)
print(b)
np.shares_memory(arr, b)

In [ ]:
arr = np.array([[0, 1, 2], [3, 4, 5]])
b = arr.T
print(arr)
print(b)
np.shares_memory(arr, b)

In [ ]:
arr = np.array([[0, 1, 2], [3, 4, 5]])
b = arr.T.reshape(1, -1)
print(arr)
print(b)
np.shares_memory(arr, b)

Analogously to the transpose case, `.reshape()` tries to minimize copying by modifying only the metadata and does not move the data in the memory. However, it is not possible to flatten a transposed array by just modifying the metadata, and a copy is needed in this case. 

Other two functions which do almost the same things are `.flatten()` and `.ravel()`. They flatten a multidimensional array into a 1D array. The difference is that `.flatten()` always returns a copy of the input array collapsed into 1D, and `.ravel()` makes a copy only when needed.

In [ ]:
arr = np.array([[0, 1, 2], [3, 4, 5]])
b = arr.flatten() # copy
print(arr)
print(b)
np.shares_memory(arr, b)

In [ ]:
arr = np.array([[0, 1, 2], [3, 4, 5]])
b = arr.ravel() # view
print(arr)
print(b)
np.shares_memory(arr, b)

## Example: Euclidian distance matrix



This is another problem we will see also later during the workshop.

Starting with a set of points points

$$x_1, x_2,\ldots, x_n \in \mathbb{R}^k,$$

the elements of their [Euclidean distance matrix](https://en.wikipedia.org/wiki/Euclidean_distance_matrix) $A$ are given by squares of pair-wise distances between them.

$$A_{ij} = \text{dist}(x_i, x_j)^2 = \text{d}_{i,j}^2 = \|x_i - x_j\|^2,$$

where $\|p\|$ denotes the [Euclidean norm](https://en.wikipedia.org/wiki/Norm_(mathematics)#Euclidean_norm) on $\mathbb{R}^k$

$$
\|p\| = \sqrt{p_1^2 + \ldots + p_k^2}
$$

<math>\begin{align}A = \begin{bmatrix}
0 & d_{12}^2 & d_{13}^2 & \dots & d_{1n}^2 \\
d_{21}^2 & 0 & d_{23}^2 & \dots & d_{2n}^2 \\
d_{31}^2 & d_{32}^2 & 0 & \dots & d_{3n}^2 \\
\vdots&\vdots & \vdots & \ddots&\vdots&  \\
d_{n1}^2 & d_{n2}^2 & d_{n3}^2 & \dots & 0 \\
\end{bmatrix}\end{align} </math>

The task is to compute  $A$, given a set of $n$ $k$-dimensional points.

In [ ]:
def setup_points(n):
    """create n points in 3d for testing"""
    points = []
    for i in range(0, 3 * n, 3):
        points.append((float(i), 1 + float(i // 2), 1 + float(i // 3)))
    return points


def dist_squared(a, b):
    return (a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2 + (a[2] - b[2]) ** 2


def dist_matrix(points):
    rows = []
    for p in points:
        row = []
        for q in points:
            row.append(dist_squared(p, q))
        rows.append(row)
    return rows

In [ ]:
points = setup_points(4)
for i, row in enumerate(points):
    print("point", i, ":", row)
print()

M = dist_matrix(points)

print("pairwise distances:")
for row in M:
    print(row)

In [ ]:
points = setup_points(1000)
%timeit M = dist_matrix(points)

**Runtime complexity**: $O(n^2)$ which we cannot improve because we have to iterate over all $n^2$ pairs of points.

### First optimization

To make the code faster we try to use NumPy to get rid of one or both of the Python loops involved and to replace number wise computations by vectorization.

In [ ]:
points = np.array(setup_points(3))

print("points matrix with shape", points.shape)
print(points)

p0 = points[0, :]
print("\nfirst point with shape", p0.shape)
print(p0)
print()

Although `points` and `p0` have different dimensions, the expression `points - p0` is defined and can be computed using NumPy broadcasting:

In [ ]:
print("broadcasting in action: matrix minus first row")
print(points - p0)
print()

Result: the $i$-th row of `points - p0` is the difference of $p_i$ and $p_0$. 
To compute the squared distance we have to square values and then add them up:

In [ ]:
print("squared values")
print((points - p0) ** 2)
print()

print("horizontal sum")
print(((points - p0) ** 2).sum(axis=1))
print()

The last vector is the first row of the distance matrix!

**The horizontal sum computed the squared distance of $p_0$ to all other points!**

To compute the full distance matrix  we loop `p0` over all rows or the points matrix:

In [ ]:
def dist_matrix_broadcast(points):
    points = np.array(points)
    result = []
    for p0 in points:
        distances_to_p0 = ((points - p0) ** 2).sum(axis=1)
        result.append(distances_to_p0)
    return np.array(result)


print(dist_matrix_broadcast(points))

Let's compare timings:

In [ ]:
p = setup_points(1_000)

%timeit dist_matrix(p)
%timeit dist_matrix_broadcast(p)

**THIS IS ABOUT 20 times faster!!!**



### Can we do better?

There is still one Python loop involved! Can we get rid of this by using broadcasting or vectorization also?

The following solution is now **more advanced**, we will not explain this in detail since there is a bit more math involved and you might want to read this and work out the details later.

**Trick**: apply a generalization of the 2nd binomial formula

$$
\mathrm{dist}(a, b)^2 = \| a - b \|^2 =\|a\|^2 - 2 \left<a, b\right> +  \|b\|^2
$$ 

where $\left< a, b \right>$ is the scalar product of the vectors $a$ and $b$. This leads to the following implementation:

In [ ]:
def dist_matrix_no_for_loop(points):
    # To avoid computing the same multiple times, use:
    # <x-y, x-y> = <x,x> -2 <x,y> + <y,y>

    points = np.array(points)
    p_2 = (points ** 2).sum(axis=1)

    # Note: The following is a slightly more efficient way
    # of writing (points**2).sum(axis=1):
    # p_2 = np.einsum("ij,ij->i", points, points)

    x_2 = p_2[:, None]
    y_2 = p_2[None, :]

    # @ is matrix multiplication, afterwards
    # entry (i, j) of x_y is <p_i, p_j> !!!
    x_y = points @ points.T

    # broadcast x_2 of shape (n, 1), x_y of shape (n, n)
    # and y_2 of shape (1, n):
    return x_2 - 2 * x_y + y_2

In [ ]:
# check for correctness
print(dist_matrix_no_for_loop(setup_points(3)))

In [ ]:
p = setup_points(1_000)

%timeit dist_matrix(p)
%timeit dist_matrix_broadcast(p)
%timeit dist_matrix_no_for_loop(p)

So we gained another factor of 4-5!!!

**You found the last example difficult to read and understand?**

We warned you in section 1 already that optimized code can be complicated and hard to read!

<img src="images/4t7moh.jpg" width=20%/>

Last but not least SciPy also offers a function to compute distance matrices, this function is completely implemented in C:

In [ ]:
from scipy.spatial.distance import cdist


def dist_matrix_scipy(points):
    return cdist(points, points, "sqeuclidean")


%timeit dist_matrix_scipy(p)

Let us run a final runtime analysis to compare all four approaches:

In [ ]:
import time

import matplotlib.pyplot as plt


def measure_time(method, n):
    p = setup_points(n)
    times = []
    for _ in range(3):
        s = time.time()
        method(p)
        times.append(time.time() - s)
    return min(times)


def measure_and_plot_times(ns, method, style):
    times = [measure_time(method, n) for n in ns]
    plt.plot(ns, times, style, label=method.__name__)

In [ ]:
ns = (10, 100, 500, 750, 1000, 1500, 2000, 3000, 4000)

plt.figure(figsize=(9, 6))
measure_and_plot_times(ns[:6], dist_matrix, "r:")
measure_and_plot_times(ns, dist_matrix_broadcast, "g:")
measure_and_plot_times(ns, dist_matrix_no_for_loop, "b:")
measure_and_plot_times(ns, dist_matrix_scipy, "y:")

plt.legend();

**Conclusion**: Runtime complexity of all three implementations is $n^2$ but with significantly different factors / constants!



## NumExpr

> NumExpr is a fast numerical expression evaluator for NumPy. With it, expressions that operate on arrays (like `3*a+4*b`) are accelerated and use less memory than doing the same calculation in Python.
> In addition, its multi-threaded capabilities can make use of all your cores – which generally results in substantial performance scaling compared to NumPy.
> Last but not least, NumExpr can make use of Intel’s VML (Vector Math Library, normally integrated in its Math Kernel Library, or MKL). This allows further acceleration of transcendent expressions.
<table>
    <tr><td><center><sub>Source: <a href="https://pypi.org/project/numexpr/">pypi.org/project/numexpr/</a></sub></center></td></tr>
</table>

Documentation: [numexpr.readthedocs.io](https://numexpr.readthedocs.io/en/latest/)

**Features**
* Avoids temporary memory copy / allocations
* Is more clever to avoid cache misses on cpu (see Sec 1)

Lets review the Euclidean distance example (NumPy version):

In [ ]:
import numpy as np


def dist_matrix_no_for_loop(points):
    points = np.array(points)
    p_2 = np.einsum("ij,ij->i", points, points)
    x_2 = p_2[:, None]
    y_2 = p_2[None, :]
    x_y = points @ points.T
    return x_2 - 2 * x_y + y_2

In the last line the expression `x_2 - 2 * x_y + y_2` involves 3 memory allocations internally, as NumPy computes this expression in several intermediate steps:

1. Compute `tmp_1 = 2 * x_y`
2. Compute `tmp_2 = x_2 - tmp_1`
3. Compute `result = tmp_2 + y_2`.

For each intermediate result, a new vector will be allocated and the result of the arithmetic operation will be stored into it.

The library NumExpr solves this problem. It can compile a vector expression into code involving only one memory allocation (in the case of an inline operation even zero).

In [ ]:
import numexpr
import numpy as np


def dist_matrix_numexpr(points):
    points = np.array(points)
    p_2 = np.einsum("ij,ij->i", points, points)
    x_2 = p_2[:, None]
    y_2 = p_2[None, :]
    x_y = points @ points.T  # @ is matrix multiplication
    return numexpr.evaluate("x_2 - 2*x_y + y_2")

In [ ]:
points = setup_points(10_000)
%timeit dist_matrix_no_for_loop(points)
%timeit dist_matrix_numexpr(points)

We now also compare the memory consumption of both versions. We will only compare the part which differs between the two versions and prepare the variables used there first:

In [ ]:
%load_ext memory_profiler

points = np.array(points)

p_2 = np.einsum("ij,ij->i", points, points)
x_2 = p_2[:, None]
y_2 = p_2[None, :]
x_y = points @ points.T  # @ is matrix multiplication

In [ ]:
%memit -r 10  x_2 - 2 * x_y + y_2

In [ ]:
%memit -r 10  numexpr.evaluate("x_2 - 2*x_y + y_2")

So, using NumExpr, we get some small improvement in performance and can reduce memory usage.

## Summary

* How to outsource 🐌 `for` loops to 🚀 low level libraries
* How to avoid unnecessary memory allocations (temporaries)

<table>
    <tr><td><img src="images/and-now-for-something-completely-different.jpg" width="400px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://www.themoviedb.org/movie/9267-and-now-for-something-completely-different">www.themoviedb.org</a></sub></center></td></tr>
</table>

> For a Schwarzschild black hole (a black hole with no rotation or electromagnetic charge), given a free fall particle starting at the event horizon, the maximum propper time (which happens when it falls without angular velocity) it will experience to fall into the singularity is `π*M` (in natural units), where M is the mass of the black hole. For Sagittarius A* (the black hole at the centre of the milky way) this time is approximately 1 minute.

> Schwarzschild black holes are also unique because they have a space-like singularity at their core, which means that the singularity doesn't happen at a specific point in *space* but happens at a specific point in *time* (the future). This means once you are inside the event horizon you cannot point with your finger towards the direction the singularity is located because the singularity happens in your future: no matter where you move, you will "fall" into it.

Source: [pythoninsider: Python 3.10 announcement](https://pythoninsider.blogspot.com/2021/10/python-3100-is-available.html)

# JIT: just-in-time compilation

Recall, that **compilation** is a process of translating computer code written in a developer-friendly high-level *source programming language* into a machine-friendly low-level *target programming language*.

Compilation of large source code may take a lot of time. How about instead of compiling the whole source code, we compile only the slow parts of the code, and only before they are actually executed for the first time? That's the main idea behind the Just-In-Time (JIT) compilation.

<dl>
    <dt><strong>Just-In-Time (JIT) compilation</strong></dt>
    <dd>For interpreted languages, like Python, it is possible to <em>compile (parts) of the source code on-demand (on their first run)</em>.
    Most JIT compilers analyze code during the first execution and created more efficient code for later executions.
    </dd>
</dl>

## How does JIT work? An illustrative example

Discalimer: **The following example is neither accurate nor real, it serves only an educational purpose**.

We implement a generic function to add up a collection of items for which `+` is defined, e.g. numbers or strings:

In [ ]:
def add_up(values):

    if len(values) < 1:
        return None

    sum_ = values[0]
    for el in values[1:]:
        sum_ += el

    return sum_

We know from the Section 1 that Python is slow here because `+` is implemented for multiple types: one can add strings, numbers or any other object that implements own addition operator.

In [ ]:
import numpy as np

ndarray_values = np.arange(100_000)

%timeit add_up(ndarray_values)

add_up(ndarray_values)

A JIT compiler might trace that this function was used with a `numpy` array and then generate optimized code on the fly. The compiler will not change your source code but use internally another implementation in memory:

In [ ]:
def add_up(values):
    if isinstance(values, np.ndarray):
        return np.sum(values)

    if len(values) < 1:
        return None

    sum_ = values[0]
    for el in values[1:]:
        sum_ += el

    return sum_

So when you call the function again you will automatically use a hopefully faster **specialized implementation**.

In [ ]:
%timeit add_up(ndarray_values)

add_up(ndarray_values)

This happens incrementally, so calling again the same function but with a list of strings:

In [ ]:
str_values = ["a", "b", "c"]

add_up(str_values)

might make JIT compiler to come up with sth more specialized for this case, like:

In [ ]:
def add_up(values):
    if isinstance(values, np.ndarray):
        return np.sum(values)

    if len(values) < 1:
        return None

    # educational purpose only: a slow per-item type check
    if all(isinstance(item, str) for item in values):
        return "".join(values)

    sum_ = values[0]
    for el in values[1:]:
        sum_ += el

    return sum(a)

In [ ]:
add_up(str_values)

In this simple example the optimizations happened on the highest level of our main function. In real life, this sum could happen nested deep in some other loops or `if`/`elif`/`else` branches.

The order of operations from the entry-point of the function to the result is called an **execution path**.

**Real JIT compilation** is much more elaborate than this previous example but the basic idea is:

- record information such as types along an execution path, and
- try to replace (part of) the execution path by a faster specialized version, based on the recorded information.

<div class="alert alert-block alert-warning">
  ⚠ In case these optimizations happen in memory they will be forgotten when you start the program again.
</div>

## PyPy

<blockquote>
  <p>.. if you want your code to just magically run faster, you should probably just use PyPy ..</p>
    <footer>
        —Guido van Rossum (creator of Python), <cite>PyCon 2015 talk</cite>
        <table>
            <tr><td><center><img src="images/guido.jpg" width="200px"></center></td></tr>
            <tr><td><center><sub>Source: <a href="https://commons.wikimedia.org/w/index.php?curid=4974869">https://commons.wikimedia.org/w/index.php?curid=4974869</a></sub></center></td></tr>
        </table>
    </footer>
</blockquote>

| | | |
|-|-|-|
| ![](images/logos/py-flat.svg) | Reference Python language implementation| C- and Python-implemented CPython interpreter (\*) |
| ![](images/logos/pypy.svg) | **[PyPy](https://www.pypy.org/) (Python in Python)** | alternative Python-implemented (\*\*) interpreter for Python code that always performs JIT-compilation |

<p style="font-size: smaller;">(*) CPython behaves like an interpreter, running directly the given source code, but it actually also performs a cached-compilation, creating intermediate bytecode - the <code>.pyc</code> files in <code>__pycache__</code> folder - which is run by CPython's virtual machine.</p>

<p style="font-size: smaller;">(**) PyPy is actually implemented in <a href="https://rpython.readthedocs.io/en/latest/">RPython (Restricted Python)</a> - a subset of Python language, which can directly be compiled to C. RPython requires e.g. that a variable's type can be inferred at compile time.</p>

**How does it work?**

PyPy interprets your code as it runs to an intermediate JIT-aware bytecode, recording meanwhile what the interpreter does. PyPy identifies the frequently executed parts of the bytecode (foremost, the often executed loops) which are JIT-compiled to a fast running machine-specific code.

<table>
    <tr><td><img src="images/PyPy_compilation.jpg" width="500px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://www.geeksforgeeks.org/difference-various-implementations-python/">https://www.geeksforgeeks.org/difference-various-implementations-python/</a></sub></center></td></tr>
</table>

<div class="alert alert-block alert-success">
ℹ You can simply try using PyPy instead of CPython to get your Python code run few times faster.
</div>

Let's try it out with the values adding up illustrative example. Let's write it to file to use the PyPy interpreter instead of the default CPython interpreter. Let's also add some extra timing prints for comparison.

In [ ]:
!python -V

In [ ]:
!pypy -V

To compare `PyPy` to `CPython` in a notebook we must write the Python code to a file and then call `pypy` resp. `python` from the command line:

In [ ]:
%%file add_up.py
import time

def add_up(values):
    t_start = time.time()

    if len(values) < 1:
        return None

    sum_ = values[0]
    for el in values[1:]:
        sum_ += el

    print(f"sum = {sum_}")
    print(f"time: {time.time() - t_start:g}s")
    print()

    return sum_

if __name__ == "__main__":
    t_start = time.time()
    int_values = list(range(10_000_000))
    for _ in range(3):
        add_up(int_values)

    print(f"total time: {time.time() - t_start:g}s")

In [ ]:
!time python add_up.py

In [ ]:
!time pypy add_up.py

Let's also compare real run time for the Euclidian distance matrix example:

In [ ]:
%pycat ../../examples/euclidean_distance.py

In [ ]:
print("CPython interpreter:")
print()
!time python ../../examples/euclidean_distance.py
print()
print("PyPy interpreter:")
print()
!time pypy ../../examples/euclidean_distance.py

[PyPy cannot re-use a previously compiled code](https://doc.pypy.org/en/latest/faq.html#couldn-t-the-jit-dump-and-reload-already-compiled-machine-codehttps://doc.pypy.org/en/latest/faq.html#couldn-t-the-jit-dump-and-reload-already-compiled-machine-code), i.e. it always JIT compiles the code, but **even with the JIT compilation overhead always added, PyPy often runs faster**.

<div class="alert alert-block alert-warning">
  ⚠ Since PyPy compiles different traces of a function, memory requirements of function objects are higher than these of CPython
</div>

In [ ]:
%%file min_mem_profile.py
from memory_profiler import profile

@profile
def temporary_array():
    return
    
if __name__ == "__main__":
    temporary_array()

In [ ]:
!pip install --user memory-profiler

In [ ]:
!pypy min_mem_profile.py

<div class="alert alert-block alert-warning">
  ⚠ Although popular "C-heavy" scientific libraries like <code>NumPy</code> and <code>SciPy</code> support modern <code>PyPy</code>, using such <a href="https://doc.pypy.org/en/latest/faq.html#do-c-extension-modules-work-with-pypy">Python C-extensions might be slower when used with PyPy</a>, mostly due to a start-up overhead
</div>

In [ ]:
%%file add_up_numpy.py
import time
import numpy as np

def add_up(values):
    t_start = time.time()

    sum_ = np.sum(values)

    print(f"sum = {sum_}")
    print(f"time: {time.time() - t_start:g}s")
    print()

    return sum_

if __name__ == "__main__":
    t_start = time.time()
    ndarray_values = np.arange(10_000_000)
    for _ in range(3):
        add_up(ndarray_values)
    print(f"total time: {time.time() - t_start:g}s")

In [ ]:
!time python add_up_numpy.py

In [ ]:
!time pypy add_up_numpy.py

<div class="alert alert-block alert-warning">
  ⚠ <a href="https://www.pypy.org/compat.html">PyPy compatiblity with Python language is restricted</a> - <strong>your code most likely won't compile out-of-the-box if</strong>
<ul>
    <li>it uses CPython extensions, which are not implemented to work with <code>PyPy</code>, most often implicitly via popular dependencies like <code>pandas</code> or <code>scikit-learn</code>,</li>
    <li>it uses modern Python features (*),</li>
    <li>...</li>
</ul>
</div>

<p style="font-size: smaller;">
(*) On 5 Aug 2022, Python stable release is 3.10, whereas <a href="https://www.pypy.org/download.html">PyPy only supports Python 3.9</a>.
</p>

<div class="alert alert-block alert-info">
    <p>ℹ&nbsp;<strong>PyPy works best with pure Python code</strong>.</p>
    <p>PyPy is a <em>low-hanging fruit</em>: easy to install, and when it works, it works well, but there is a high chance it won't work out-of-the-box with external libraries.</p>
</div>

## Numba

[Numba](https://numba.pydata.org) is a JIT compiler that translates manually designated Python functions into  machine code.

* uses explicit decorators for functions, like `@numba.jit`, to indicate JIT compilation,
* supports NumPy arrays,
* supports CPU and GPU parallelization.

**How does it work?**

Numba is built on a [LLVM compiler infrastructure](https://llvm.org/) (*), in particular on LLVM JIT compiler and on LLVM Intermediate Representation (IR). IR is the data structure or code used internally by a compiler (or a virtual machine) to represent source code.

<table>
    <tr><td><img src="images/Python_Numba_LLVM.png" width="600px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://ep2019.europython.eu/talks/hsbcAZF-understanding-numba-the-python-and-numpy-compiler/">https://ep2019.europython.eu/talks/hsbcAZF-understanding-numba-the-python-and-numpy-compiler/</a></sub></center></td></tr>
</table>
    
<p style="font-size: smaller;">(*) The LLVM compiler infrastructure is used in compilers for various languages such as <a href="https://clang.llvm.org">Clang</a> for C/C++ or <code>rustc</code> for <a href="https://www.rust-lang.org">Rust</a>.</p>

### `@numba.jit` application


Recall the example to approximate $\pi$:

In [ ]:
from random import uniform


def approx_pi(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        # check if (x, y) lies in the cirle
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

To mark and prepare a function for Numba JIT compilation, decorate it with `numba.jit` decorator:

In [ ]:
import numba


@numba.jit
def approx_pi_jit(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        # check if (x, y) lies in the cirle
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

Let us check how this small modifiation impacts run time:

In [ ]:
%timeit approx_pi(5000)
%timeit approx_pi_jit(5000)

On first run the actual LLVM JIT compilation happens and this can be relatively slow.

You see in the following example the timing differences between first and following runs:

In [ ]:
# the next two lines make sure that the previous JIT
# compilation is "forgotten" in the notebook:
del approx_pi_jit
approx_pi_jit = numba.jit(approx_pi)

print("first run")

# run exactly once:
%timeit -r1 -n1 approx_pi_jit(5000)

print()
print("second run")
%timeit -r1 -n1 approx_pi_jit(5000)

print()
print("third run")
%timeit -r1 -n1 approx_pi_jit(5000)

### *Object* and *No Python* modes
        
Numba supports compilation of only some features of the Python language; it does not support e.g. 

- dictionaries, (a Numba specific `dict` replacement is [avaible as an experimental feature](https://numba.pydata.org/numba-doc/0.43.0/reference/pysupported.html#dict)),
- generators or comprehensions (cf. [Numba's Supported Python features](https://numba.readthedocs.io/en/stable/reference/pysupported.html))
- lists of mixed types, see [here](https://numba.pydata.org/numba-doc/0.43.0/reference/pysupported.html#list) or [lists of lists](https://numba.pydata.org/numba-doc/0.43.0/reference/pysupported.html#list-reflection).

In case Numba can not compile a feature, it will use the so called **Object mode**. This mode allows your program to run, but it **does not use JIT compilation when a non-supported Python feature is encountered**. In turn, Numba might actually make your program significanty slower (as the compiled code needs to communicate back and forth with the Python interpreter).

By default when using `@numba.jit` Numba uses the Object mode. Instead, you can try to force the **No Python** mode by using `nopython=True` option, or equivalently `@numba.njit`. In the No Python mode Numba **always tries to JIT compile decorated code and triggers an error when a non-supported Python feature is encountered**.

In [ ]:
import numba


@numba.njit  # same as: @numba.jit(nopython=True)
def approx_pi_jit(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        # check if (x, y) lies in the cirle
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In [ ]:
%timeit approx_pi_jit(5000)

It's a good practice to always try to force No Python mode. You will get an error, when Numba does not understand some functions or objects, like e.g. `pandas` `DataFrame` .. but only on JIT compilation:

In [ ]:
import random
import numba
import pandas as pd

def use_pandas(nrow=1_000):
    df = pd.DataFrame(
        {
            "x": random.sample(range(nrow), k=nrow),
            "y": random.sample(range(nrow), k=nrow),
        }
    )
    return (df ** 2).sum().sum()

# alternative use of njit:
use_pandas_jit = numba.njit(use_pandas)

try: use_pandas_jit()
except Exception as e: print("💥", e)

<div class="alert alert-block alert-success">
    <b>Good practice</b>: always use <code>@numba.njit</code> instead of <code>@numba.jit</code>.
</div>

<div class="alert alert-block alert-warning">
    ⚠ When using Object mode and when Numba does not understand used objects, like e.g. the <code>pandas</code> data frames, you will get a warning, but no error, and the code won't really be faster
</div>

It might even run slower due to the Numba's overhead:

In [ ]:
# explicitly force Object mode to remove warnings
# try without forcing Object mode w/ plain @numba.jit
use_pandas_jit = numba.jit(forceobj=True)(use_pandas)

print("Python:\n", use_pandas())
%timeit -n 32 use_pandas()
print("\nNumba JIT compilation:")
%time use_pandas_jit()
print("\nNumba:\n", use_pandas_jit())
%timeit -n 32 use_pandas_jit()
print()

### numpy support

Numba seamlessly supports large part of NumPy (cf. [Supported numpy features](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html)):

In [ ]:
import numba
import numpy as np

def use_numpy():
    a = np.random.randint(0, 10, size=(1000, 2))
    return (a ** 2).sum()

# force non-Python mode
use_numpy_jit = numba.njit(use_numpy)

print("Python:")
%timeit -n 32 use_numpy()
print("\nNumba JIT compilation:")
%time use_numpy_jit()
print("\nNumba:")
%timeit -n 32 use_numpy_jit()
print()

**Vectorize = Numpy `ufunc`**

Additionally, Numba provides [`@vectorize`](https://numba.pydata.org/numba-doc/latest/user/vectorize.html#vectorize) ([`@guvectorize`](https://numba.pydata.org/numba-doc/latest/user/vectorize.html#guvectorize)) decorators to make it easy to define own NumPy (generalized) universal functions (`ufunc`):

In [ ]:
# Numba type signature for types inference;
# (cf. https://numba.pydata.org/numba-doc/latest/reference/types.html)

# here: take two 64-bit integers and return a 64-bit float
@numba.vectorize(["float64(int64, int64)"])
def divide_ints(x, y):
    if y == 0:
        if x == 0:
            return np.nan
        return np.sign(x) * np.inf
    return x / y

In [ ]:
print("scalar / scalar")
print(divide_ints(2, 2))

print("\nscalar / vector")
print(divide_ints(1, [0, 2, 1]))  # scalar / vector

print("\nvector / vector")
print(divide_ints([1, 2], [2, 1]))  # element-wise vector / vector

In [ ]:
print("\nbroadcast")
x = np.arange(3).reshape(3, 1)
y = np.arange(9).reshape(3, 3)
print(divide_ints(x, y))

print("\nufunc.outer")
print(divide_ints.outer([1, 2], [2, 3, 4]))  # all vs. all

The arguments types of Numba-vectorized functions are validated to agree with `@numba.vectorize` declaration

In [ ]:
divide_ints(2.1, 1.9)

### Parallelization

To make use of multi-core CPU architecture, add a `parallel=True` option to the decorator and, in `for` loops replace `range()` with `numba.prange()`, but **only for *embarrassingly parallel* or reduction/accumulation-based computations**.

Let's compare the serial and parallel variantes of the "numbified"  $\pi$ approximation example:

In [ ]:
from random import uniform
import numba

@numba.njit
def approx_pi_jit(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In [ ]:
@numba.njit(parallel=True)
def approx_pi_par(n_attempts):
    n_hits = 0

    for _ in numba.prange(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts


print(approx_pi_jit(2_000_000))
print(approx_pi_par(2_000_000))

In [ ]:
n = 2_000_0000

%timeit approx_pi_jit(n)
for n_threads in (2, 3, 4, 8):
    print()
    print(f"and on #threads = {n_threads}")
    print()
    numba.set_num_threads(n_threads)
    %timeit approx_pi_par(n)

<div class="alert alert-block alert-warning">
    ⚠ <code>for</code> loops are not in general easily parallelized (<it>embarassingly parallel</it>) as shown above
</div>

For instance:

In [ ]:
import numba


def fibonacci_loop(n):
    if n == 0:
        return 0
    if n == 1:
        return 1

    prev_prev_fib = 0
    prev_fib = 1
    for i in range(2, n + 1):
        fib = prev_fib + prev_prev_fib
        prev_prev_fib = prev_fib
        prev_fib = fib
    return prev_fib

In [ ]:
# WRONG use of `parallel=True` and `numba.prange`
@numba.njit(parallel=True)
def fibonacci_loop_numba_wrong(n):
    if n == 0:
        return 0
    if n == 1:
        return 1

    prev_prev_fib = 0
    prev_fib = 1
    for i in numba.prange(2, n + 1):
        fib = prev_fib + prev_prev_fib
        prev_prev_fib = prev_fib
        prev_fib = fib
    return prev_fib

print("Expecting:", fibonacci_loop(35))
print("Got:", fibonacci_loop_numba_wrong(35))

**Parallel `ufunc`s**


You can also "target" the Numba-vectorized functions for multi-core CPU or even GPU by specifying `target="parallel"` or `target="cuda"`.

For demonstration purposes, let's make the $\pi$ approximation function a `ufunc` (vectorized over number of attempts), both serial and parallel:

In [ ]:
from random import uniform
import numba


@numba.vectorize("float64(int64,)")
def approx_pi_vec(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In [ ]:
@numba.vectorize("float64(int64,)", target="parallel")
def approx_pi_vec_par(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In [ ]:
v = [2_000_000] * 4

print("#### vectorized pi approximation")
print(approx_pi_vec(v))
%timeit approx_pi_vec(v)

for n_threads in (2, 3, 4, 6, 8):
    print()
    print(f"#### vectorized pi approximation with {n_threads} threads")
    numba.set_num_threads(n_threads)
    print(approx_pi_vec_par(v))
    %timeit approx_pi_vec_par(v)
    print()

<div class="alert alert-block alert-warning">
    ⚠ parallelization of Numba-vectorized <code>ufunc</code> parallelization has some shortcomings, e.g. <a href="https://github.com/numba/numba/issues/1721">producing wrong results when using <code>ufunc.accumulate</code> or <code>ufunc.reduce</code> on a Numba-vectorized parallel function</a>.
</div>

**Threading Layers**

[Numba's Threading Layers](https://numba.pydata.org/numba-doc/latest/user/threading-layer.html) are the backend libraries that perform the parallel execution; except for the default built-in task scheduler, Numba supports **Intel TBB** and **OpenMP** parallel programming libraries.

You can get the threading_layer with

```python
from numba import threading_layer; @numba.njit(parallel=True) ...; print(threading_layer())
```

See [here](https://numba.pydata.org/numba-doc/latest/user/threading-layer.html#the-threading-layers)

### Good to know

Options:

* [`cache=True`](https://numba.pydata.org/numba-doc/dev/user/jit.html#cache) - save compiled code to share with later Python interpreter invocations (note: this is caching, of a compiled function code, and not memoization, of function return values; cf. Section 3)
* [`fastmath=True`](https://numba.pydata.org/numba-doc/latest/user/performance-tips.html#fastmath) - trade accuracy (IEEE-defined compliance, e.g. loop operations order) for even more speed
* [`nogil=True`](https://numba.pydata.org/numba-doc/dev/user/jit.html#jit-nogil) - release GIL e.g. in functions that explicitly manage threads

Decorators:

* [`@jitclass`](https://numba.pydata.org/numba-doc/latest/user/jitclass.html#jitclass) - create a JIT-aware classes (early version)
* [`@cfunc`](https://numba.pydata.org/numba-doc/latest/user/cfunc.html#cfunc) - create a C callback to use in an external C library
* [`@cuda.jit`](https://numba.pydata.org/numba-doc/latest/cuda/kernels.html) - NVIDIA CUDA kernel to run on GPUs (will be shown later)
* [`@stencil`](https://numba.pydata.org/numba-doc/latest/user/stencil.html#numba-stencil) - array neighborhood computations (stencil-like operation kernel)

### Exercise 1 [10 min]

*Numbify* the the NumPy variant of the $\pi$ hit'n'miss approximation example. Which of the Numba, NumPy, `numpy+Numba` variants performs best? How about after multi-core parallelization?

In [ ]:
import numpy as np

def approx_pi_numpy(n_attempts):
    points = np.random.rand(n_attempts, 2)
    n_hits = np.count_nonzero((points ** 2).sum(1) <= 1.0)
    return 4 * n_hits / n_attempts

%timeit approx_pi(5000)
%timeit approx_pi_numpy(5000)

### Exercise 2 [10 min]

*Numbify*, in the *No Python* mode, the NumPy variant of the `dist_matrix` function from the Euclidian distance matrix example. How does the runtime compare with respect to the original variant? How about when using `fastmath=True` option?

*Hint: not all NumPy features are supported, e.g. `numpy.einsum` is not supported, or you need to use `expand_dims` or `reshape` instead of indexing/slicing with `None`.*

In [ ]:
# %load ../examples/euclidean_distance_numpy.py
#!/usr/bin/env Python3
"""Euclidean distance example in 3 dimensional space"""

import numpy

def setup_points(n):
    return numpy.indices((n, n, n), dtype=float).reshape((3, -1)).T

def dist_matrix(points):
    # To avoid computing the same multiple times, use:
    # <x-y, x-y> = <x,x> -2 <x,y> + <y,y>
    # The following is a more efficient way of writing (points**2).sum(axis=1):
    p_2 = numpy.einsum("ij,ij->i", points, points)
    x_2 = p_2[:, None]
    y_2 = p_2[None, :]
    x_y = points @ points.T  # @ is matrix multiplication
    return x_2 - 2 * x_y + y_2

if __name__ == "__main__":
    M = dist_matrix(setup_points(10))
    print(M[:5, :5])

## Summary

* How to use pypy (🚀) instead of CPython (🐌) if this is possible
* Use the `@numba.njit` decorator to just-in-time-compile python code

**And Now For Something Completely Different**

<table>
    <tr><td><img src="images/and-now-for-something-completely-different-2.jpg" width="500px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://madmovieman.com/wp-content/uploads/2013/04/And-Now-For-Something-Completely-Different.jpg">madmovieman.com</a></sub></center></td></tr>
</table>

<blockquote>
In mathematics, a Borwein integral is an integral whose unusual properties were first presented by mathematicians David Borwein and Jonathan Borwein in 2001. These integrals are remarkable for exhibiting apparent patterns that eventually break down. The following is an example:



$$
\int_0^\infty \frac{\sin(x)}{x} dx = \frac{\pi}{2}
$$
$$
\int_0^\infty \frac{\sin(x)}{x} \frac{\sin(x/3)}{x/3} dx = \frac{\pi}{2}
$$
$$
\int_0^\infty \frac{\sin(x)}{x} \frac{\sin(x/3)}{x/3} \frac{\sin(x/5)}{x/5}dx = \frac{\pi}{2}
$$
This pattern continues up to
$$
\int_0^\infty \frac{\sin(x)}{x} \frac{\sin(x/3)}{x/3} \cdots \frac{\sin(x/13)}{x/13}dx = \frac{\pi}{2}.
$$
At the next step the obvious pattern fails,
$$
\int_0^\infty \frac{\sin(x)}{x} \frac{\sin(x/3)}{x/3} \cdots \frac{\sin(x/15)}{x/15}dx = \frac{467807924713440738696537864469}{935615849440640907310521750000}\pi
$$
$$
= \frac{\pi}{2} - \frac{6879714958723010531}{935615849440640907310521750000}
\approx \frac{\pi}{2} - 2.31 \times 10^{-11}.
$$

</blockquote>

Source: [pythoninsider: Python 3.9.1 announcement](https://blog.python.org/2020/12/python-391-is-now-available-together.html)

# AOT: ahead-of-time compilation

<dl>
    <dt><strong>Ahead-Of-Time (AOT) compilation</strong></dt>
    <dd>Compilation in languages, like C/C++, is done <em>before the program is run at all</em>. The bigger the source code to be compiled the longer it takes to compile it.</dd>
</dl>

## Pythran

[Pythran](https://github.com/serge-sans-paille/pythran) is a *low-hanging fruit* AOT C++ compiler for Python, designed for scientific Python programs.

In particular Pythran has **no** support for:

* classes (or object-oriented programming techniques in general),
* containers (`list`, `dict`, ...) with inhomogeneous element types (except for tuples),
* non-overflowing `int`s (only numbers that fit 64 bit integers; `int64_t`).

Also Pythran uses deep copies, except for `numpy.ndarray`.
E.g. when you modify a list argument within a function, the caller of the function will not see the changes.

cf. [Pythran disclaimer](https://pythran.readthedocs.io/en/latest/MANUAL.html#disclaimer) and [Pythran limitations](https://pythran.readthedocs.io/en/latest/MANUAL.html#limitations).

We will use Pythran IPython cell magic extension to run AOT compilation of the code in the cells.

In [ ]:
import pythran

%load_ext pythran.magic

Pythran make use of type annotations of the function signatures; we add them to the code as comments:
```python
# pythran export function_name(arg1_type, arg2_type, ...)
```

In [ ]:
%%pythran

from random import uniform


# pythran export approx_pi_pythran(int)
def approx_pi_pythran(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In [ ]:
%timeit approx_pi(2_000_000)
%timeit approx_pi_pythran(2_000_000)

Analogous to C++ compilers, you can also use Pythran compiler with options to:

* optimize code (`-O`),
* use your native CPU's microprocessor level *single instruction, multiple data* (SIMD) support ([XSIMD](https://github.com/xtensor-stack/xsimd)), or
* seamlessly parallelize `for` loops (using OpenMP)

Note: except the `-O3` flag in the following example, the flags `-DUSE_XSIMD` and `-fopenmp` work on Linux but not on Mac.

In [ ]:
%%pythran -O3 -DUSE_XSIMD -fopenmp

from random import uniform


# pythran export approx_pi_pythran_optimized(int)
def approx_pi_pythran_optimized(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In [ ]:
%timeit approx_pi_pythran_optimized(2_000_000)

BUT Pythran has also (partial) NumPy support, and, when it does work, it does work fast with NumPy code:

In [ ]:
import numpy


def approx_pi_numpy(n_attempts):
    points = numpy.random.rand(n_attempts, 2)
    n_hits = numpy.count_nonzero((points ** 2).sum(1) <= 1.0)
    return 4 * n_hits / n_attempts

In [ ]:
%timeit approx_pi_numpy(2_000_000)

In [ ]:
%%pythran
# %load ../examples/pi_numpy.py
#!/usr/bin/env Python3

import numpy


# pythran export approx_pi_numpy_pythran(int)
def approx_pi_numpy_pythran(n_attempts):
    points = numpy.random.rand(n_attempts, 2)
    n_hits = numpy.count_nonzero((points ** 2).sum(1) <= 1.0)
    return 4 * n_hits / n_attempts

In [ ]:
%timeit approx_pi_numpy_pythran(2_000_000)

## Summary
* Use pythran to compile restricted Python code => 🚀

# Cython


<div class="alert alert-block alert-warning">
    <a href="https://cython.readthedocs.io/en/latest/src/userguide/migrating_to_cy30.html">This section will be outdated as soon as Cython 3.0 will be out.</a>
</div>

<div class="alert alert-block alert-success">
    Another similar aproach, which does not extend the Python syntax is
    <a href="https://github.com/python/mypy/tree/master/mypyc">mypyc</a>
</div>

* Language extension to Python allowing compilation of Python code to binary Python extensions.
* `cdef` keyword to add static type C-like declarations, so the code (`.pyx` file extension) can be translated to C first (`.c` file). 
* In a second step, the `.c` file is compiled using a C compiler like `gcc` to a `.so` (unix) or a `.pyd` (windows) shared library.

Code of many low level languages can be compiled into such shared libraries (C, C++, Fortran, ...).
The resulting shared library will be built with an entrypoint for the Python interpreter, and can directly be imported from pure Python code.

`.pyx` files do not necessarily have to use Cython type annotations. Cython also compiles pure Python files. However the resulting extension will not as optimized as when using annotated code.

**Usecases:**
* Optimization of Python code
* Wrap C / C++ libs: https://github.com/eth-cscs/PythonHPC/tree/master/cython/wrapping_c

We start with pure 🐍 code: the $\pi$ example.

In [ ]:
from random import uniform

def approx_pi(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In [ ]:
%time approx_pi(2_000_000)

In [ ]:
%timeit approx_pi(2_000_000)

Lets see what happens if we start turning this code into Cython code! In the following, we are going to use the **jupyter notebook** [extension](https://cython.readthedocs.io/en/latest/src/quickstart/build.html#jupyter).
If you want to use Cython natively (i.e. outside of a notebook), one way is to let a `setup.py` script compile the code for you. See [here](https://cython.readthedocs.io/en/latest/src/quickstart/build.html#building-a-cython-module-using-setuptools).

Another way (which only works if you are not using any C extensions) is to use `pyximport`.
This is for building Cython modules during development without explicitly running `setup.py` after each change.
E.g. if you have a `pi.pyx`, you could do

```python
import pyximport; pyximport.install()
import pi
...
```

For more details, see the [Cython docs](https://docs.cython.org/en/latest/src/userguide/source_files_and_compilation.html#pyximport).

In [ ]:
%load_ext Cython

In [ ]:
%%cython

from random import uniform


def approx_pi_cython(n_attempts):
    n_hits = 0

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In [ ]:
%time approx_pi_cython(2_000_000)

In [ ]:
%timeit approx_pi_cython(2_000_000)

Although it is an improvement, we did not fully take the advantage of Cython.
To work with maximal performance, Cython needs to know the type of every variable (is the variable a `float` or is it an `int`). This is similar to NumPy (see Section 1.1.1).



We add some type declarations and things will get better. **Type declarations** are defined by the `cdef` keyword (old style / not pure Python) or using types from the `cython` module, e.g. `cython.int` (modern style).
We will first introduce the old style.

<div class="alert alert-block alert-warning">
    As types for the variables, we will use <code>long</code> here. The Python type <code>int</code> is implemented by a <code>long</code> in C. If you would use a C <code>int</code>, you would get a integer overflow for integers greater than $2147483647 = 2^{31}$.
</div>

In [ ]:
%%cython

from random import uniform


def approx_pi_cython_v2(n_attempts):
    cdef long n_hits = 0 # added type
    cdef double x, y # added types

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In [ ]:
%time approx_pi_cython_v2(2_000_000)

In [ ]:
%timeit approx_pi_cython_v2(2_000_000)

So we gained almost a factor of 1.5 by adding some type declarations.

Can we improve speed further?

If you add `--annotate` or `-a` to view the generated C code, yellow lines will mark parts of the code which could not be converted to pure C code and require fall back to interpreted Python.

<div class="alert alert-block alert-success">
    By minimizing the yellow part, you maximize Cython's performance!
</div>

In the following example Cython cannot figure out a types for `n`, and `i`. Therefore it uses a generic CPython type, all lines performing operations involving those variables will be marked yellow:

In [ ]:
%%cython --annotate

from random import uniform


def approx_pi_cython_v2(n_attempts):
    cdef long n_hits = 0
    cdef double x, y

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

Note: you can also click on the lines to see the generated `C` code!

You can see that the code behind the yellow lines involving the Python interpreter (e.g. line 9) looks very complicated, whereas the other code, e.g. line 12, looks much simpler, although the original variable names are mangled into `C` variable names.

Lets try to get rid of the yellow line containing the for loop:

In [ ]:
%%cython --annotate
from random import uniform


def approx_pi_cython_v3(long n_attempts): # added type
    cdef long n_hits = 0
    cdef double x, y

    for _ in range(n_attempts):
        x = uniform(-1.0, 1.0)
        y = uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

Thats all we can do by just adding type hints.

* The yellow line at `import` does not matter, it is only called once.
* We cannot improve the lines at `def` and `return`. By removing the Python interaction at `def` and `return`, the function becomes inaccessable from pure Python code. The interactions are responsible for converting data to and from `C`.
* We also cannot remove the Python interaction originating from the `uniform` call, as this is a pure Python function. This line is in fact the real bottle neck of the performance.

In [ ]:
%timeit approx_pi_cython_v3(2_000_000)

To optimize further, we have to replace the Python random generator by a C equivalent.

A low level function which only takes C variables as input and only returns C variables (cannot be used in pure python code), can be declared by using `cdef` followed by the C return type instead of `def`:

In [ ]:
%%cython --annotate
cimport cython

cdef extern from "stdlib.h":
    double drand48()
    void srand48(long int seedval)
cdef extern from "time.h":
    long int time(int)
cdef double c_uniform(double x0, double x1):
    return x0 + drand48() * (x1 - x0)
srand48(time(0)) # Initialize random number generator

def approx_pi_cython_v4(long n_attempts):
    cdef long n_hits = 0
    cdef double x, y

    for _ in range(n_attempts):
        x = c_uniform(-1.0, 1.0)
        y = c_uniform(-1.0, 1.0)
        if x ** 2 + y ** 2 <= 1.0:
            n_hits += 1
    return 4 * n_hits / n_attempts

In this [appendix](./appendix.ipynb) you can read more on low level C functions.

In [ ]:
print("Pure 🐍")
%time approx_pi(2_000_000)
print("Cython - untyped")
%time approx_pi_cython(2_000_000)
print("Cython - partially typed")
%time approx_pi_cython_v2(2_000_000)
print("Cython - typed")
%time approx_pi_cython_v3(2_000_000)
print("Cython - Fully optimized")
%time approx_pi_cython_v4(2_000_000)

🚀 So we are now faster by a factor of about ~120.

If you manage to get rid of all Python interaction, you even can expect speedup by a factor up to ~100 depending on the problem.

## Using Python type hints
Instead of using `cdef` we can use Python type hints. However, if we would just use pure Python types hints (e.g. `int`), Cython would ignore them. The reason is that Python only offers `int` as an integer type, where in C/Cython there are more than just one integer type (e.g. `short`, `int`, `long`). Instead, we have to use Cython counterparts (i.e. `cython.int`, `cython.long` instead of `int`).

<div class="alert alert-block alert-warning">
     ⚠ Types as <code>int</code>, <code>double</code>, ... are actually defined in both modules <code>cython</code> and <code>Cython</code>. Only types from <code>cython</code> (lower case) will work!
</div>

In [ ]:
%%cython --annotate

import cython

def approx_pi_lattice_cython_v3(delta: cython.double):  # Changed type hint
    n_hits: cython.long = 0 # Changed type hint
    x: cython.double # Changed type hint
    y: cython.double # Changed type hint
    x = -1.
    while x <= 1.:
        y = -1.
        while y <= 1.:
            if x ** 2 + y ** 2 <= 1.0:
                n_hits += 1
            y += delta
        x += delta
    return 4 * n_hits * delta**2

### Exercise (20 min)

The following version of the $\pi$ example,
we dont use a random generator, but cover the square by a lattice instead.

Rewrite this version using Cython!

In [ ]:
def approx_pi(resolution):
    """
    :param resolution: number of grid -1 points in 1 dimension
    """
    n_hits = 0
    for i in range(resolution):
        for j in range(resolution):
            x = 2 * i / resolution - 1
            y = 2 * j / resolution - 1
            if x ** 2 + y ** 2 <= 1.0:
                n_hits += 1
    return 4 * n_hits / resolution ** 2

**Step 1**: Just use type annotations (prefer pure Python type hints, e.g. `cython.int` over `cdef`) to reduce the yellow lines.

Why are the lines `x = 2*i/resolution - 1` and `y = 2*i/resolution - 1` still yellow?

Hint: If you cant figure it out, [look here](./exercise_hints.ipynb).

**Step 2**: Get rid of the Python interaction at the two yellow lines mentioned in step 1. If you dont know how, read the hint in step 1.

## Auto infer types inside functions
When working with functions, instead of explicitly give type annotations, Cython also can **automatically infer types** via a decorator `@infer_types`:

In [ ]:
%%cython --annotate

import cython

@cython.infer_types
def approx_pi_lattice_cython_v4(delta: cython.double):  # Still have to give a type hint here
    n_hits = 0 # automatically inferred
    x = -1. # automatically inferred
    while x <= 1.:
        y = -1. # automatically inferred
        while y <= 1.:
            if x ** 2 + y ** 2 <= 1.0:
                n_hits += 1
            y += delta
        x += delta
    return 4 * n_hits * delta**2

Note, that we still have to declare the type of the argument `delta` in the function declaration.

## numpy support
Cython can handle NumPy data types out of the box:

In [ ]:
%%cython --annotate
import numpy

def sum_array(double[:] x):
    cdef int i, nx = x.shape[0]
    cdef double s = 0.0
    for i in range(nx):
        s += x[i]
    return s

The notation `double[:]` is inspired by numpy.
Here, it declares a 1-dim array. You would use `double[:, :]` to declare a 2-dim array, and so on.

For safety reasons, Cython checks if we try to access elements out of the array boundaries. Furthermore it allows using negative array indices and translates them to positive indices (counting back from the end).
Both **safety features may be disabled to improve performance**:

In [ ]:
%%cython --annotate
from cython cimport boundscheck, wraparound


@wraparound(False)
@boundscheck(False)
def sum_array(double[:] x):
    cdef int i, nx = x.shape[0]
    cdef double s = 0.0
    for i in range(nx):
        s += x[i]
        
    return s

## Parallelization with OpenMP

MP abbreviates *Multi Processing*. It is an API for multi-platform shared-memory multiprocessing programming on one machine.
Cython can make use of OpenMP.

In the following cell, we are injecting the compiler args `-fopenmp` via distutils (distutils is used behind the scenes to start the compiler) to compile and link the block with the OpenMP C library.

In [ ]:
%%cython --annotate

# distutils: extra_compile_args = -fopenmp -march=native
# distutils: extra_link_args = -fopenmp
# cimport numpy  # special compile-time information about the numpy module
from cython cimport boundscheck, wraparound
from cython.parallel cimport prange


@boundscheck(False)
@wraparound(False)
def parallel_sum(double[:, :] x, double[:] out):
    cdef int i, j, m = x.shape[0], n = x.shape[1]
    for i in prange(m, nogil=True):
        for j in range(n):
            out[i] += x[i, j]
    return out

## Advanced features
Have a look at the [Cython tutorials](https://cython.readthedocs.io/en/latest/src/tutorial/index.html).
E.g.
* [C-like Allocation/Dealllocation](https://cython.readthedocs.io/en/latest/src/tutorial/memory_allocation.html)
* [Extension Types](https://cython.readthedocs.io/en/latest/src/tutorial/cdef_classes.html)

## Packaging

<div class="alert alert-block alert-success">
    <b>Good practice</b>: <a href="https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#distributing-cython-modules">Distribute the generated .c files alongside the Cython sources</a>, so that users are not required to have Cython available.
    <p>Also Cython compilation should be disabled by default during setup.</p>
</div>

Even if the end user has Cython installed, he/she probably doesn’t want to use it just to install the module.
Also, the installed Cython version may differ from the Cython version used to package, and may not compile your sources correctly.

An example of a folder structure for the `euclidean_distance` package would be

```
euclidean_distance
├── euclidean_distance.pyx
├── euclidean_distance.c
├── pyproject.toml
├── README.md
└── setup.py
```

`euclidean_distance.pyx`: Cython code

`README.md`: Description / docs about your package

`euclidean_distance.c`: Generated by Cython, enclosed to the distribution

`pyproject.toml` declares the build environment (without it `pip install` would not know which build system to use to install the package)

```toml
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"
```

ℹ If you dont include the `.c` file, also `cython` would be required, and you would have to add `"Cython"` to `requires`.

`setup.py`: Compile recipy.

Could look like:

```python
#!/usr/bin/env Python3

from numpy import get_include as _numpy_get_include
from setuptools import setup, Extension

try:
  from Cython.Build import Cythonize
  USE_CYTHON = True
except ImportError:
  USE_CYTHON = False

ext = '.pyx' if USE_CYTHON else '.c'
extensions = [Extension(
  "euclidean_distance",
  sources=["euclidean_distance" + ext],
  include_dirs=[_numpy_get_include()])]

if USE_CYTHON:
  extensions = Cythonize(extensions)

setup(
  name='euclidean_distance',
  ext_modules=extensions,
  zip_safe=False,
  install_requires=["numpy"]
)
```

With this package structure, you can build the package for direct usage in the current project folder, using

```bash
python setup.py build_ext --inplace
```

To distribute it (both binary and source versions) to pypi, you would use

```bash
python setup.py sdist bdist_wheel
twine upload --repository-url $PYPI_REPOSITORY_URL dist/*
```

Where you have to specify the url to your pypi project. For more details, see [here](https://twine.readthedocs.io/en/latest/)

## Summary

* How to modify pure Python code (🐌) to Cython code (🚀)
* Compile Cython code to a Python C extension
* Packaging to a Python extension

<table>
    <tr><td><img src="images/and-now-for-something-completely-different-3.jpg" width="500px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://www.themoviedb.org/movie/9267-and-now-for-something-completely-different/images/posters?image_language=en">www.themoviedb.org</a></sub></center></td></tr>
</table>


# Using other compiled languages

<div class="alert alert-block alert-info">
    ℹ When developing extensions written in compiled languages, you will need to install a compiler for that language.
</div>

## Fortran: F2PY (part of numpy)
WTF is Fortran?

![Fortran](./images/hpc-joke.svg)

<small>Fortran (Formula Translation) is a compiled imperative programming language designed for numeric computation and scientific computing. In the number crunching scene, [it recently again gained popularity](https://www.zdnet.com/article/this-old-programming-language-is-suddenly-getting-more-popular-again/).</small>

<div class="alert alert-block alert-success">
    <a href="https://numpy.org/doc/stable/f2py/index.html">F2py</a> comes bundled with numpy (<code>pip install numpy</code>).
    Fortran code that works fine with F2py is very easy to integrate!
</div>

F2py turns Fortran code into a Python extension.

It uses numpy_distutils that supports a number of Fortran 77/90/95 compilers, including GNU, Intel, Sun Fortre, SGI MIPSpro, Absoft, NAG, Compaq etc. compilers.
For more details, see [here](https://numpy.org/doc/stable/f2py/f2py.getting-started.html#three-ways-to-wrap-getting-started).


Here is the Monte Carlo $\pi$ example in Fortran:

`pi.f`:
```Fortran
      SUBROUTINE APPROX_PI(N_ATTEMPTS, OUT)
C
C     MONTE CARLO COMPUTATION TO APPROXIMATE PI
C     
      INTEGER N_ATTEMPTS
      REAL*8 OUT
      REAL*8 X, Y
Cf2py intend(in) n_attempts
Cf2py intent(out) out
      INTEGER N_HITS

      N_HITS = 0
      
      CALL SRAND(86456)

      DO I=1,N_ATTEMPTS
         X = 2.0D0 * RAND() - 1.0D0
         Y = 2.0D0 * RAND() - 1.0D0
         IF (X*X + Y*Y <= 1.0D0) THEN
            N_HITS = N_HITS + 1
         ENDIF
      ENDDO
      OUT = 4.0D0 * N_HITS / N_ATTEMPTS
      END
```

To build a Python extension, just use
```bash
python -m numpy.f2py -c -m pi pi.f
```

Then use it in a Python 🐍 file:
```python
from pi import approx_pi

print(approx_pi(2_000_000))
```

To deal with numpy arrays, you have to make sure, that input arrays are Fortran-contiguous, i.e. matrix data is transposed in memory compared to the C version (see the [docs](https://numpy.org/doc/stable/f2py/f2py.getting-started.html#the-quick-way)).

## C/C++

### [ctypes](https://docs.python.org/3/library/ctypes.html)

Foreign function library

* part of the Python Standard Library
* Provides C compatible data types
* Allows loading and calling functions in existing shared libraries (see [cython introduction](#Cython))
* Can be used to wrap libraries in pure Python
* No need for compilation, if shared C library already is available

Monte Carlo $\pi$ example in C:

`approx_pi.c`:
```C
#include <stdlib.h>

#include "approx_pi.h"

double approx_pi(unsigned int n_attempts) {
  /* C core function   */
  unsigned int n_hits = 0, i;
  double x, y;
  for(i=0; i<n_attempts; i++) {
    x = 2.*rand()/RAND_MAX -1.;
    y = 2.*rand()/RAND_MAX -1.;
    if(x*x + y*y <= 1.0) n_hits ++;
  }
  return 4. * n_hits / n_attempts;
}
```

Compile it with `gcc -shared approx_pi.c -o approx_pi.so`.

Use it in 🐍:
```python
from ctypes import cdll, c_double

approx_pi = cdll.LoadLibrary("./approx_pi.so").approx_pi
approx_pi.restype = c_double  # By default ctypes assumes int as a return type

pi = approx_pi(2_000_000)
print(pi, type(pi))
```

### [pybind11](https://pybind11.readthedocs.io/en/stable/), [boost Python](https://www.boost.org/doc/libs/1_79_0/libs/python/doc/html/index.html), [nanobind](https://github.com/wjakob/nanobind)

* Useful for big projects
* You can even map C++ classes to Python classes

Monte Carlo $\pi$ example in pybind11 (C++):

`pi.cpp`:
```cpp
#include <random>
#include <pybind11/pybind11.h>

double approx_pi(unsigned int n_attempts) {
  std::random_device generator;
  std::uniform_real_distribution<double> distribution(-1.0,1.0);
  
  unsigned int n_hits = 0;
  double x, y;
  for(unsigned int i=0; i<n_attempts; i++) {
    x = distribution(generator);
    y = distribution(generator);
    if(x*x + y*y <= 1.0) n_hits ++;
  }
  return 4. * n_hits / n_attempts;
}

PYBIND11_MODULE(pi, m) {
    m.doc() = "Pi"; // optional module docstring

    m.def("approx_pi", &approx_pi, "Approximates pi");
}
```

Compile it with
```bash
c++ -O3 -Wall -shared -std=c++11 -fPIC `python3 -m pybind11 --includes` pi.cpp -o pi`python3-config --extension-suffix`
```

Use it in 🐍:
```python
from pi import approx_pi

print(approx_pi(2_000_000))
```

### [SWIG](http://swig.org/)

ℹ SWIG also can be used for other programming languages than C

Monte Carlo $\pi$ example: Take `approx_pi.c` from Subsection *ctypes*.
Add a file `pi_swig.i`:

```c
%module pi_swig

%{
#define SWIG_FILE_WITH_INIT
#include "approx_pi.h"
%}

double approx_pi(unsigned int n_attempts);
```

Then let swig autogenerate C code to wrap `approx_pi.c` into a C extension, using Pythons C-API:
```bash
swig -python pi_swig.i
```

Write a `setup.py`:

```python
#!/usr/bin/env Python3
"""
To only compile the C extension inplace:
python setup.py build_ext --inplace
"""

try:
    from setuptools import setup, find_packages, Extension
except ImportError:
    raise RuntimeError('setuptools is required')

setup(name="pi_swig",
      version="1.0",
      packages=find_packages(),
      package_dir={"pi_swig": "."},
      description="Pi",
      Python_requires=">=3.5",
      ext_modules=[Extension(
          '_pi_swig',
          sources=["approx_pi.c", "pi_swig_wrap.c"])]
      )
```

Compile it with `setup.py build_ext --inplace`.

Use it in 🐍:
```python
from pi_swig import approx_pi

print(approx_pi(2_000_000))
```

## [Rust](https://www.rust-lang.org/)

* Main focus: memory / thread safety without the need of a garbage collector
* Near C performance
* High level abstractions like C++
* Low boilerplate 🐍 extensions with the [PyO3](https://pyo3.rs) crate (Rust package)

As Rust (Abbreviated by 🦀 because of Rust's mascot, Ferris the Crab) is heavily based on its own ecosystem (cargo as build system / dependency manager), the developpers created an own 🐍 packager [Maturin](https://github.com/PyO3/maturin) (also available on pypi) which can replace setuptools for 🦀 extensions.

Monte Carlo $\pi$ example in 🦀:

`src/lib.rs`
```rust
use pyo3::prelude::{pymodule, PyModule, PyResult, Python};
use rand;

#[pymodule]
fn pi(_py: Python, m: &PyModule) -> PyResult<()> {
    #[pyfn(m, "approx_pi")]
    fn approx_pi<'a>(_py: Python, n_attempts: u32) -> PyResult<f64> {
        let mut n_hits: u32 = 0;
        for _ in 0..n_attempts {
            let x = 2. * rand::random::<f64>() - 1.;
            let y = 2. * rand::random::<f64>() - 1.;
            if x * x + y * y <= 1.0 {
                n_hits += 1;
            }
        }
        Ok(4. * n_hits as f64 / n_attempts as f64)
    }
    Ok(())
}
```

To build it with cargo, you also need a `Cargo.toml` file:
```toml
[package]
name = "pi"
version = "0.1.0"
authors = ["Chuck Norris <chucknorris@roundhouse.gov>"]
edition = "2021"

[lib]
name = "pi"
crate-type = ["cdylib"]

[dependencies]
pyo3 = {version = "0.12", features = ["extension-module"]}
rand = "0.7"
```

Build it with `cargo build --release`  (here, we are not using maturin). Copy the resulting `target/release/libpi.so` as `pi.so` into either your `$PYTHON_PATH` or into the directory, your 🐍 importing this extension resides.

Use it in 🐍:
```python
from pi import approx_pi

print(approx_pi(2_000_000))
```

## Summary

* Basics on how to use foreign languages (🚀) in python (🐌)

# HPC techniques: When to use what?
In general: stick with numpy as long as you can, occasionally bringing in Numba!

* Sometimes there is an existing library written in Fortran / C you want to wrap -> use Cython / f2py / pyO3 / ...
* Sometimes there is a usecase not covered by NumPy / SciPy. Numba not fast enough or you also want to have interfaces to another language -> Use a foreign language

|  |  |  |  |
|--|--|--|--|
| ![](images/logos/py-flat.svg) | Pure Python | easy but 🐌 |
| ![](images/logos/numpy.svg) | numpy | Easy to use but restricted, good algorithms out of the box. Use as long as you can! |
| ![](images/logos/numba.svg) | numba | Less restricted than numpy, some care to distinguish between code that can be handled by Numba (not everything supported), array processing |
| | Pythran | Works on restricted Python only |
| ![](images/logos/cython.svg) | Cython | <table>    <tr><td>-</td><td>Need to learn new Cython syntax</td></tr>    <tr><td>-</td><td>More work to setup for the compilation / setup.py, code preparation</td></tr>    <tr><td>+</td><td>Flexibility (even more than NumPy / Numba)</td></tr>    <tr><td>-</td><td>C Pitfalls (SEG Fault)</td></tr>    <tr><td>+</td><td>Use C libraries in Python</td></tr>    <tr><td>-</td><td>Problematic on Windows because of not standard windows C compiler, WSL</td></tr>    <tr><td>-</td><td>Not pure Python (e.g. <code>cdef</code> keyword) -> lock in (will hopefully change soon in Cython 3.0)</td></tr></table> |
| ![](images/logos/C.svg) ![](images/logos/fortran.svg) ![](images/logos/rust.svg) | Foreign Languages | <table><tr><td>-</td><td>Need to know that language</td></tr><tr><td>-</td><td>need a compiler / adds overhead in the distribution phase</td></tr><tr><td>+</td><td>maximal flexibility / 🚀</td></tr></table> |

ℹ Cython and foreign languages require compilers to be installed. This adds a layer of difficulty, because e.g. this is not the case on Windows by default (but currently there is a free installer from MS to get a compiler). Therefore in this case, it is good practice to build and distribute binary wheels in addition to source packages.

# References

* [Python as glue](https://numpy.org/doc/1.18/user/c-info.python-as-glue.html)
* [numexpr](https://numexpr.readthedocs.io/en/latest/)
* [PyPy](https://www.pypy.org/)
* [Numba](https://numba.pydata.org/)
* [Pythran](https://github.com/serge-sans-paille/pythran)
* [Cython](https://cython.org/)
* [F2PY](https://numpy.org/doc/stable/f2py/index.html)
* [ctypes](https://docs.python.org/3/library/ctypes.html)
* [pybind11](https://pybind11.readthedocs.io/en/stable/)
* [Swig](http://swig.org/)
* [PyO3](https://pyo3.rs)